## Problem Statement

Let's give it a crack! After reading the introduction, it seems to me the goal of this project is to prediction the result of the match between two teams, based on the 'knowledge' we have learnt about the teams. Therefore, it is important to grab as much information as possible from the past records of the teams.

## Data inspection

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

teams = pd.read_csv('./input/teams.csv')
seasons = pd.read_csv('./input/seasons.csv')

In [7]:
# make team_id <==> team_name dict
team_dict = {}
for x, y in zip(teams['Team_Id'], teams['Team_Name']):
    team_dict[x] = y
print('Total number of teams: {}'.format(len(team_dict)))

Total number of teams: 364


In [3]:
regular_season = pd.read_csv('./input/RegularSeasonCompactResults.csv')
regular_season_detail = pd.read_csv('./input/RegularSeasonDetailedResults.csv')

In [4]:
import re
t = list(map(lambda x: re.search('W*', x).group(), regular_season_detail.columns.tolist()))

In [5]:
Tourney = pd.read_csv('./input/TourneyCompactResults.csv')
Tourney_detail = pd.read_csv('./input/TourneyDetailedResults.csv')
Tour_seed = pd.read_csv('./input/TourneySeeds.csv')
Tour_slots = pd.read_csv('./input/TourneySlots.csv')

With all the information loaded, how to build a sensible model? The ideal model, like a crystal ball, should takes in two teams, and then split out the winning team. This should be a very backbone of the model. However, in reality, there are more factors that we might to consider: for example, how much games has the each team played prior to their encounter? How fresh are their legs? Are they historical rivals (like Duke and UNC). Also, although we have many years of historical data available, the roster of a college basketball team changes on a yearly basis, therefore, the histroical record 10 years ago might not be as useful as the record last year.

Well, other than the names of the two teams, what else should we provide to the crystal ball? Most likely, the crystal ball needs to know more about each team, such as avarge points scored/allowed per game, win/loss record for the season, average rebounds per game, etc. These attributes, for each team, can be found from the provided training data. 

## Feature Engineering

From above thinking, we need to prepare a set of features for each of the two teams, and then basing these features can we make a prediction. What are the features we can distill from the existing data, and more important, what are the features are important?

The rule of the game is simple: you need to outscore your oppoent to win. Therefore, a good indicator will be the average point per game, let's call it *ppg*, if team A has a higher ppg than team B, and this is the only information I have, I would bet team A would beat team B. Now the question is, how do you get the ppg information? For the first game in the season, where can I get this number, maybe use the average ppg from last season? For the last game in the season, does it make more sense to use the average ppg from the previous games in the season? Therefore, we need to take this information into account.

Let's start building such feature(s) for each team, and for each regular season. Given the fact that the statisitic after 2003 (*RegularSeasonDetailedResults.csv*) have more information than the years before (*RegularSeasonCompactResults.csv*), let me start from 2003 season


In [31]:
team_stat = teams # get the team id and names
stats = [
         'fgm', # field goal made
         'fga', # field goal attempt
         'fgm3', # 3-pointer made
         'fga3', # 3-pointer attempted
         'ftm', # free-throw made
         'fta', # free-throw attempt
         'or', # offensive rebound
         'dr', # defensive rebound
         'ast', # assists
         'to', # turnover
         'stl', # steals
         'blk', # blocks
         'pf', # personal fouls
        ]
new_stats = [
         # ===== below are new features =====
         'ppg', # point per game
         'oppg', # oppent point per game (new feature)
         'margin', # point margin (new feature)
         'fgp', # field goal percentage (new feature)
         'fgp3', # 3-pointer percentage (new feature)
         'ftp', # free-throw percentage (new feature)
         'odr', # offensive-defensive rebound ratio (new feature)
         'otw', # total overtime wins, not averaged! (new feature)
         'otl', # total overtime loses, not averaged! (new feature)
        ]
# get the season averaged stat
df_win = regular_season_detail.groupby('Wteam').sum()
df_lose = regular_season_detail.groupby('Lteam').sum()

In [30]:
df_win

,Season,Daynum,Wscore,Lteam,Lscore,Numot,Wfgm,Wfga,Wfgm3,Wfga3,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
Wteam,,,,,,,,,,,,,,,,,,,,,
1101,36277,1362,1406,22343,1246,0,472,954,140,326,...,255,313,464,181,390,178,274,102,57,371
1102,399752,12572,13549,260060,10874,7,4579,9227,1655,4064,...,3748,1962,2913,1985,3883,2085,2942,1048,523,3770
1103,586907,22382,21789,365653,18054,26,7492,16107,2321,6118,...,5049,3767,5550,3415,6256,3101,4372,1760,672,5771
1104,508380,16898,18527,323729,15261,17,6465,13856,1601,4432,...,5014,2715,4097,3062,5333,2700,3599,1434,771,4831
1105,267227,11937,9730,166224,8338,8,3250,7520,831,2420,...,2221,1817,2948,1814,3199,1409,2295,924,409,2828
1106,389832,17257,13617,240763,11638,15,4723,10565,1147,3238,...,3054,2613,4045,2186,4128,1860,2864,1292,554,3986
1107,462414,16186,16365,294797,13785,13,5545,11968,1522,3898,...,4577,2457,3663,2245,4631,2565,3079,1375,727,4544
1108,216992,9808,7774,134024,6884,14,2670,5761,621,1665,...,1885,1546,2437,1261,2299,1087,1537,734,278,2151
1110,472217,17482,16137,290414,13647,22,5523,11620,1669,4110,...,4232,2663,3923,2204,4772,2540,3050,1393,584,4536
